Random code, ignore this

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

from fxvol.backtest import run_backtest
from fxvol.data_utils import load_csv, save_csv
from fxvol.fin_comp import qlike_loss, realized_vol
from fxvol.baseline_models import (
    ewma_forecast,
    garch11_forecast,
    har_forecast,
    naive_forecast,
    rolling_mean_forecast,
)

In [2]:
# Data

HORIZON = 5
log_ret = load_csv("processed", "log_returns")
log_ret = log_ret["EUR"]

In [3]:
from fxvol.data_utils import make_features, make_xy

In [4]:
X = make_features(log_ret=log_ret, window = HORIZON, lags = [1, 5], use_asym=True)

In [15]:
X

,lr,rv,rv_5,asym
Date,,,,
2010-01-14,-0.001654,0.004393,0.005359,0.004393
2010-01-15,-0.007540,0.005558,0.005440,0.005558
2010-01-18,0.001094,0.004084,0.005066,0.000000
2010-01-19,-0.008764,0.005274,0.004995,0.005274
2010-01-20,-0.012023,0.005368,0.004935,0.005368
...,...,...,...,...
2025-12-16,0.001445,0.002931,0.002872,0.000000
2025-12-17,-0.000435,0.002762,0.003015,0.002762
2025-12-18,-0.000611,0.001646,0.002688,0.001646


In [18]:
X.shift(-1)

,lr,rv,rv_5,asym
Date,,,,
2010-01-14,-0.007540,0.005558,0.005440,0.005558
2010-01-15,0.001094,0.004084,0.005066,0.000000
2010-01-18,-0.008764,0.005274,0.004995,0.005274
2010-01-19,-0.012023,0.005368,0.004935,0.005368
2010-01-20,-0.000945,0.005512,0.005159,0.005512
...,...,...,...,...
2025-12-16,-0.000435,0.002762,0.003015,0.002762
2025-12-17,-0.000611,0.001646,0.002688,0.001646
2025-12-18,-0.001420,0.001050,0.002287,0.001050


In [6]:
X, y = make_xy(log_ret=log_ret,horizon = HORIZON, lags = [1, 5], use_asym=True)

In [13]:
pd.concat([X,y], axis=1).tail(10)

,lr,rv,rv_5,asym,y
Date,,,,,
2025-12-09,0.000105,0.002052,0.001787,0.000000,0.002931
2025-12-10,-0.001128,0.002047,0.001917,0.002047,0.002762
2025-12-11,0.006252,0.003277,0.002283,0.000000,0.001646
2025-12-12,0.003258,0.003057,0.002494,0.000000,0.001050
2025-12-15,-0.000153,0.003046,0.002696,0.003046,0.001188
2025-12-16,0.001445,0.002931,0.002872,0.000000,0.002690
2025-12-17,-0.000435,0.002762,0.003015,0.002762,0.002836
2025-12-18,-0.000611,0.001646,0.002688,0.001646,0.002869
2025-12-19,-0.001420,0.001050,0.002287,0.001050,0.002807


In [ ]:
exog.iloc[-1]

In [ ]:
real_vol

In [ ]:
exog

In [ ]:
eur_ret.isna().sum()

In [ ]:
real_vol.isna().sum()

In [ ]:
exog.isna().sum()

In [ ]:
from arch import arch_model

In [ ]:
scaled_ret = 100*eur_ret
am = arch_model(scaled_ret, vol="GARCH", p=1, o=0, q=1, dist="normal")

In [ ]:
res = am.fit(update_freq=5, disp='off')

In [ ]:
(res.forecast(horizon=5).variance)**0.5

In [ ]:
100*real_vol.iloc[-1]

In [ ]:
res.forecast(horizon=horizon).mean.iloc[-1, -1] / 100

In [ ]:
models = [
    # (naive_forecast, "naive", {}),
    # (rolling_mean_forecast, "rolling5", {"window": 5}),
    # (rolling_mean_forecast, "rolling20", {"window": 20}),
    # (rolling_mean_forecast, "rolling50", {"window": 50}),
    # (rolling_mean_forecast, "rolling100", {"window": 100}),
    # (ewma_forecast, "ewma092", {}),
    # (ewma_forecast, "ewma030", {"alpha": 0.3}),
    # (har_forecast, "har1-5-22", {"lags": [1, 5, 22]}),
    # (har_forecast, "har1-5-22-66", {"lags": [1, 5, 22, 66]}),
    (garch11_forecast, "garch11", {}),
]

# Run backest

HORIZON = 5

scores = pd.DataFrame(
    index=[model[1] for model in models], columns=["RMSE", "MAE", "QLIKE"]
)

for forecast_fn, name, params in models:
    results = run_backtest(
        log_ret=eur_ret, forecast_fn=forecast_fn, horizon=HORIZON, stride=100, **params
    )
    y_true = results["y_true"]
    y_pred = results["y_pred"]
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    qlike = qlike_loss(y_true, y_pred)
    scores.loc[name] = [rmse, mae, qlike]

save_csv(scores.astype(float).round(5), "results", "baselines")

In [ ]:
results

In [ ]:
from arch.univariate import HARX

In [ ]:
har = HARX(real_vol, lags=[1, 5, 22])

In [ ]:
res = har.fit()

In [ ]:
res.forecast(horizon=5).mean

In [ ]:
har

In [ ]:
har.fit()

In [ ]:
df_res = run_backtest(eur_ret, EWMA(0.3), horizon=HORIZON)
y_true = df_res["y_true"]
y_pred = df_res["y_pred"]

In [ ]:
from fxvol.data_utils import save_csv

save_csv(df_res.round(3), "results", "baselines")

In [ ]:
df_res.round(3)

In [ ]:
df_res.iloc[-200:].plot(y=["y_true", "y_pred"])

In [ ]:
plt.rcParams["figure.figsize"] = [15, 8]

In [ ]:
# use darts plotting style
from darts import set_option

set_option("plotting.use_darts_style", True)

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
from darts import TimeSeries

In [ ]:
from fxvol.data_utils import load_csv

historic_vol = load_csv("processed", "historic_vol")
historic_vol.dropna(inplace=True)

In [ ]:
historic_vol.index = pd.to_datetime(historic_vol.index)

In [ ]:
series = TimeSeries.from_dataframe(historic_vol[["EUR"]], freq="D")

In [ ]:
from statsmodels.tsa.stattools import adfuller

adf_result = adfuller(historic_vol[["EUR"]], maxlag=100, regression="ctt")

In [ ]:
# p-value
adf_result[1]
# Small -> No unit root, i.e. stationary series

In [ ]:
eur = historic_vol[["EUR"]].diff().dropna()

In [ ]:
adf_result = adfuller(eur, maxlag=100, regression="ctt")
adf_result[1]

In [ ]:
from darts.utils.missing_values import fill_missing_values

series = fill_missing_values(series)

In [ ]:
series.plot()

In [ ]:
train, val = series.split_before(pd.Timestamp("20200101"))
train.plot(label="training")
val.plot(label="validation")

In [ ]:
from darts.metrics import mape

In [ ]:
from darts.models import AutoARIMA, ExponentialSmoothing, Theta


def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print(f"model {model} obtains MAPE: {mape(val, forecast):.2f}%")


eval_model(ExponentialSmoothing())
eval_model(AutoARIMA())
eval_model(Theta())

In [ ]:
from darts.models import AutoARIMA, ExponentialSmoothing, Theta

In [ ]:
model = AutoARIMA()
model.fit(train)
pred = model.predict(len(val))

In [ ]:
train.plot(label="training")
val.plot(label="validation")
pred.plot(label="prediction")

In [ ]:
hist_for = model.historical_forecasts(
    series=series,
    start=0.8,
    val_length=21,
    forecast_horizon=21,
    stride=21,
    verbose=True,
)

In [ ]:
_, val80 = series.split_after(0.80)

In [ ]:
val80.plot()
hist_for.plot()

In [ ]:
print(mape(val80, hist_for))

In [ ]:
from darts.models.forecasting.baselines import NaiveMovingAverage, NaiveDrift

In [ ]:
model = NaiveMovingAverage(input_chunk_length=1)
# model = NaiveDrift()
model.fit(train)
pred = model.predict(len(val))

In [ ]:
hist_for = model.historical_forecasts(
    series=series, start=0.8, forecast_horizon=21, stride=21, verbose=True
)

In [ ]:
val80.plot()
hist_for.plot()

In [ ]:
print(mape(val80, hist_for))

In [ ]:
from darts.models import ARIMA

In [ ]:
from statsmodels.tools.sm_exceptions import ConvergenceWarning

warnings.simplefilter("ignore", ConvergenceWarning)

In [ ]:
gs = ARIMA.gridsearch(
    {"p": [1, 5, 20], "d": [0], "q": [1]},
    series=series,
    forecast_horizon=21,
    stride=210,
    verbose=True,
    show_warnings=False,
    start=0.5,
)

In [ ]:
model = gs[0]

In [ ]:
hist_for = model.historical_forecasts(
    series=series,
    start=0.8,
    val_length=21,
    forecast_horizon=21,
    stride=210,
    verbose=True,
)

In [ ]:
print(mape(val80, hist_for))

In [ ]:
val80.plot()
hist_for.plot()